In [1]:
import pandas as pd
import numpy as np
import json
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import requests
from requests_oauthlib import OAuth1
from os.path import expanduser
import time
home = expanduser("~")

In [2]:
tweets_data_path = '/Users/nathanbackblaze/DS/Metis/projects/Final Projects/AWS_data/fetched_tweets_gopro_6_22.txt'
results = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        results.append(tweet)
    except:
        continue

In [3]:
len(results)

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/nathanbackblaze

14071

The twitter api returns an array of hastags, let's organize this into a set of all the hashtags

In [4]:
from collections import Counter

In [5]:
def get_hashtags(results):
    count = Counter()
    for result in results:
        try:
            result_hashtags = result['entities']['hashtags']
            for hashtag in result_hashtags:
                count[hashtag['text'].lower()] += 1
        except:
            pass
    return count

In [6]:
hashtags_count = get_hashtags(results)
hashtags_count.most_common()

[(u'gopro', 2762),
 (u'fijipro', 276),
 (u'camera', 209),
 (u'goskateboardingday', 175),
 (u'video', 149),
 (u'deals', 108),
 (u'liveme', 96),
 (u'onairnow', 96),
 (u'3kcashprize', 94),
 (u'mount', 86),
 (u'\u30a2\u30af\u30b7\u30e7\u30f3\u30ab\u30e0', 84),
 (u'photography', 81),
 (u'ebay', 77),
 (u'travel', 76),
 (u'scuba', 76),
 (u'kit', 72),
 (u'hawaii', 72),
 (u'virales', 72),
 (u'dog', 71),
 (u'lipsync', 67),
 (u'giveaway', 63),
 (u'goprohero4', 59),
 (u'win', 56),
 (u'awesome', 53),
 (u'harness', 53),
 (u'sharkdiving', 52),
 (u'shark', 52),
 (u'periscope', 49),
 (u'hero4', 46),
 (u'hero', 44),
 (u'prize', 43),
 (u'goprooftheday', 43),
 (u'diving', 41),
 (u'videos', 41),
 (u'clamp', 41),
 (u'ebayus', 41),
 (u'competition', 38),
 (u'pug', 37),
 (u'sports', 32),
 (u'jaws', 32),
 (u'fun', 32),
 (u'usa', 32),
 (u'drone', 32),
 (u'dogs', 31),
 (u'tecnolog\xeda', 31),
 (u'teamjl', 30),
 (u'fetch', 30),
 (u'goprogirl', 29),
 (u'tech', 28),
 (u'selfie', 28),
 (u'waterproofcamera', 28),
 (u

In [7]:
df = pd.DataFrame.from_dict(hashtags_count,orient='index').reset_index()
df.rename(columns={'index':'hashtag', 0:'count'},inplace=True)
df.sort_values(by='count',ascending=False)

,hashtag,count
2297,gopro,2762
1775,fijipro,276
1996,camera,209
1790,goskateboardingday,175
1949,video,149
1934,deals,108
1425,liveme,96
1435,onairnow,96
508,3kcashprize,94
1077,mount,86


In [8]:
df.head()

,hashtag,count
0,colomitos,1
1,yellow,2
2,woods,1
3,guideschool,1
4,melianassaubeach,1


In [9]:
def get_hash_corpus(row):
    row = row.lower()
    corpus = u''
    for result in results:
        hashtag_list = []
        try:
            for sub_tag in result['entities']['hashtags']:
                hashtag_list.append(sub_tag['text'].lower())
            if row in hashtag_list:
                corpus += result['text']
        except:
            pass
    return corpus

In [10]:
df['corpus'] = [get_hash_corpus(row) for row in df['hashtag']]

In [11]:
df.head()

,hashtag,count,corpus
0,colomitos,1,Look at may Dab bitch! üòé\n#DabSquad #Colomito...
1,yellow,2,#anotherdayattheoffice #3Dprinted a #3way #360...
2,woods,1,The true beauty of Nature üçÉ\n#climbing #climb...
3,guideschool,1,#Repost @david1995smith with @repostapp\n„Éª„Éª„Éª\n...
4,melianassaubeach,1,Nassau Bahamas üå¥‚òÄÔ∏èüòé#melianassaubeach #goproh...


In [12]:
df.tail()

,hashtag,count,corpus
2505,instawork,1,Gass nggolek takjill üòá naik javmuscle üòÜ #toy...
2506,summertrip,2,Sunset NYC! gopro #hero4session #goproapp #bab...
2507,freemusic,3,Music for https://t.co/LY8vRWn23U #royaltyf...
2508,billyknowmates,1,@johnpopham @PaulBromford @John_A_Wade I'm sti...
2509,socialmarketing,1,#networkmarketing #socialmarketing #GoPro http...


We now have a data frame where the 'hashtags' column has all the hastags mentioned in the budlight twitter query, and 'corpus' has all the text from all the tweets containing the respective hashtag.

This will essentially tell us which text is used in relation to each hashtag, we can now use tfidf and Latent Semantic Analysis to find common topics

For this portion I choose to use the Textacy library (which is built on spaCy) for a few reasons:
- Textacy/spaCy run in Cython, so it is high-performing.  This import because the application will eventually be user-facing
- Textacy/spaCy can handle odd terms such as emojis
- Textacy has an implementation for TFIDF (among other features), which might be import for filtering out unimport words in the terms matrix
- Textacy has Latent Semantic Analysis built it

In [13]:
import textacy

/Users/nathanbackblaze/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [14]:
docs = df['corpus']

In [15]:
corpus = textacy.TextCorpus.from_texts('en',docs)

In [16]:
corpus

TextCorpus(2510 docs; 232743 tokens)

In [17]:
doc_term_matrix, id2term = corpus.as_doc_term_matrix((doc.as_terms_list(words=True, ngrams=(2,3), named_entities=True)for doc in corpus),weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)

In [18]:
print(repr(doc_term_matrix))

<2510x18000 sparse matrix of type '<type 'numpy.float64'>'
	with 82679 stored elements in Compressed Sparse Row format>


Now that the tfidf matrix has been, built, we can do topic modeling to see if we can gain any insights on our groups of tweeters.  At this point we can choose between Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), and Non-negative Matrix Factorization (NMF).  Here is the breakdown for why I will use LSA:

- NMF is generally suited for non text data such as pixels, and NMF can return incoherent topics
- LDA provides a topical distribution for each topic and term and often is more accurate than LSA
- LSA performs Singular Value Decomposition (SVD), which reduces the dimensionality of our already extremely large matrix.  LSA will also execute more quickly which is again important for the end user

In [19]:
model = textacy.tm.TopicModel('lsa', n_topics=10)

In [20]:
model.fit(doc_term_matrix)

In [22]:
doc_topic_matrix = model.transform(doc_term_matrix)

In [23]:
print(doc_topic_matrix.shape)

(2510, 10)


In [24]:
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'gopro   $   11.99   camera   hero4   video   gopro $   hero   13.99   @gopro')
('topic', 1, ':', u'11.99   gopro $   $   gopro $ 11.99   $ 11.99   dog   13.99   harness   fetch   pet')
('topic', 2, ':', u'hero4   gopro hero4   camera   silver   hero4 silver   gopro hero4 silver   244.00   edition   hero   gopro hero')
('topic', 3, ':', u'scuba   diving   diva   padi   hawaii   sea   mar   underwater   scubadiving   buceo')
('topic', 4, ':', u'selfie   @gopro   texas   deftfam   @xgame   turnup   ion   @xgame @gopro   turnuptuesday   austin')
('topic', 5, ':', u'drone   quad   fpv   dji   13.99   fly   https://t.co/2iaa1y3exc   atlantic\u2026 https://t.co/2iaa1y3exc   lipol   turnigy')
('topic', 6, ':', u'13.99   $ 13.99   gopro $ 13.99   clamp   flex   adjustable   great   adjustable neck   mount with adjustable   neck')
('topic', 7, ':', u'socialmedia   countryside   djlife   landrover   editing   denondj   route   videoshoot   wecreatetheworldwelivein   dj')
('top

For the above we used 10 topics so that we can show what is happening in a human readable format, but for the next steps we will want to use more topics

In [25]:
model = textacy.tm.TopicModel('lsa', n_topics=100)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
print(doc_topic_matrix.shape)

(2510, 100)


Now that we have our reduced matrix, we can do some clustering to see what hashtags are related

Because we don't want the user to have to choose the number of clusters, we have three clustering options from Sklearn that don't require a n_clusters paramter:
- DBSCAN generally performs better with more data
- MeanShift might perform well but is too computationally intensive
- Affinity Prograpagation is the most realistic option here

In [26]:
from sklearn.cluster import AffinityPropagation
ap = AffinityPropagation()
%time ap.fit(doc_topic_matrix)
clusters = ap.labels_.tolist()

CPU times: user 24.2 s, sys: 587 ms, total: 24.7 s
Wall time: 25 s


Now that we have the list of clusters, let's connect them with our hashtags to see what hashtags are being grouped together

In [27]:
df['cluster'] = clusters

In [29]:
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['cluster']]

In [59]:
df.head()

,hashtag,count,corpus,cluster,cluster_count
0,taxi,2,4 #bottle #pepsi ....\n#gari meri #taxi ....\n...,39,15
1,tachira,11,Protesta de trabajadores de la #Pepsi #Tachira...,0,33
2,agency,1,#Pepsi starts their #own #ContentMarketing #Ag...,1,3
3,summer,1,What cooler than being cool? Ice cold! \n\n#Pe...,2,4
4,esme,1,RT @karen73984451: #StaffySunday #Red &amp; Ma...,3,3


In [46]:
df.sort_values(['cluster_count'],ascending=False).groupby(["cluster"]).head()


,hashtag,count,corpus,cluster,cluster_count
1012,sponsorship,1,@eves_liam #gopro #sponsorship,1,2766
787,kyt,1,#kyt #gopro https://t.co/uWuBZKM4pB,1,2766
285,shopyelton,1,All smiles with willzoodsma and his gopro @liq...,1,2766
2297,gopro,2762,M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...,1,2766
6,cyprus,1,Œ†Œ±ŒπœáŒΩŒØŒ¥ŒπŒ± œÉœÑŒ∑ Œ∏Œ¨ŒªŒ±œÉœÉŒ± ŒºŒµ œÑŒø ŒΩŒ≠Œø Œ±ŒæŒµœÉŒøœÖŒ¨œÅ œÑŒ∑œÇ G...,1,2766
1934,deals,108,#Deals #Camera Bow Archery Rifle Gun Fishing R...,365,571
1620,art,22,#timelapse on my @gopro of #MatthewMandala doi...,365,571
2240,photography,81,Follow RT @bgamegiveaway to win a @GoPro Hero\...,365,571
1996,camera,209,https://t.co/2Dz0KfUMwX #GoPro Great accessory...,365,571
1994,usa,32,HOT #USA Deals : https://t.co/ooPMUi01b7 GoPro...,365,571


In [51]:
group = df.groupby('cluster')['count'].aggregate(sum).sort_values(ascending=False)
group

cluster
1      2766
365     571
89      352
416     340
193     335
60      293
264     289
380     286
366     286
163     207
214     202
97      162
91      138
363     131
387     124
8       118
268     111
364     111
239     108
164     107
308     100
180      97
286      96
347      94
316      84
230      83
78       82
191      71
219      71
346      70
       ... 
154       1
64        1
379       1
427       1
340       1
69        1
122       1
330       1
72        1
74        1
335       1
420       1
311       1
140       1
325       1
332       1
41        1
90        1
259       1
179       1
139       1
349       1
326       1
415       1
87        1
125       1
375       1
52        1
174       1
50        1
Name: count, dtype: int64

In [53]:
top_clusters = group[:20]
top_clusters

cluster
1      2766
365     571
89      352
416     340
193     335
60      293
264     289
380     286
366     286
163     207
214     202
97      162
91      138
363     131
387     124
8       118
268     111
364     111
239     108
164     107
Name: count, dtype: int64

In [69]:
for i in top_clusters.index:
    print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_clusters.ix[i])
    print df[['hashtag','count','corpus']][df.cluster == i].sort_values('count',ascending=False)[:10]
    print

Cluster #: 1 Total Tweeters: 2766
          hashtag  count                                             corpus
2297        gopro   2762  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...
6          cyprus      1  Œ†Œ±ŒπœáŒΩŒØŒ¥ŒπŒ± œÉœÑŒ∑ Œ∏Œ¨ŒªŒ±œÉœÉŒ± ŒºŒµ œÑŒø ŒΩŒ≠Œø Œ±ŒæŒµœÉŒøœÖŒ¨œÅ œÑŒ∑œÇ G...
285    shopyelton      1  All smiles with willzoodsma and his gopro @liq...
787           kyt      1                #kyt #gopro https://t.co/uWuBZKM4pB
1012  sponsorship      1                     @eves_liam #gopro #sponsorship

Cluster #: 365 Total Tweeters: 571
            hashtag  count                                             corpus
1996         camera    209  https://t.co/2Dz0KfUMwX #GoPro Great accessory...
1934          deals    108  #Deals #Camera Bow Archery Rifle Gun Fishing R...
2240    photography     81  Follow RT @bgamegiveaway to win a @GoPro Hero\...
1240         videos     41  #gopro #video #ebay New in box GoPro HERO 4  B...
1833         ebayus     41  #gopro #video 

Let's try Affinity Propogation again, but this time using the precomputed cosine similarity matrix

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
%time dist = 1 - cosine_similarity(doc_topic_matrix)
ap = AffinityPropagation(affinity='precomputed')
%time ap.fit(dist)
clusters = ap.labels_.tolist()
df['cluster'] = clusters
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['cluster']]
group = df.groupby('cluster')['count'].aggregate(sum).sort_values(ascending=False)
top_clusters = group[:20]
for i in top_clusters.index:
    print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_clusters.ix[i])
    print df[['hashtag','count','corpus']][df.cluster == i].sort_values('count',ascending=False)[:10]
    print

CPU times: user 165 ms, sys: 49.9 ms, total: 215 ms
Wall time: 176 ms
CPU times: user 24.3 s, sys: 746 ms, total: 25.1 s
Wall time: 27.9 s
Cluster #: 15 Total Tweeters: 3465
               hashtag  count  \
2297             gopro   2762   
1949             video    149   
2062           virales     72   
1472           lipsync     67   
2504        goprohero4     59   
2048         goprogirl     29   
1793  harmonyoftheseas     26   
653              skate     19   
962             sunset     12   
1979           lustige     11   

                                                 corpus  
2297  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
1949  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
2062  @24HorasMx Espectacular #Video GoPro: Danny Ma...  
1472  After being away for two weeks, I was afraid w...  
2504  #Gopro\n.\n.\n.\n.\n.\n.\n.\n #goprophotograph...  
2048  RT @GoPro: Photo of the Day! #GoProGirl @Letic...  
1793  RT @StephBeTravel: Kayaking in Mallorca,

Let's try this all again with DBSCAN

In [71]:
from sklearn.cluster import DBSCAN
dbs = DBSCAN()
%time dbs.fit(doc_topic_matrix)
clusters = dbs.labels_.tolist()
df['cluster'] = clusters
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['cluster']]
group = df.groupby('cluster')['count'].aggregate(sum).sort_values(ascending=False)
top_clusters = group[:20]
for i in top_clusters.index:
    if i == -1:
        pass
    else:
        print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_clusters.ix[i])
        print df[['hashtag','count','corpus']][df.cluster == i].sort_values('count',ascending=False)[:10]
        print

CPU times: user 2.62 s, sys: 90.2 ms, total: 2.71 s
Wall time: 2.89 s
Cluster #: 0 Total Tweeters: 8333
                 hashtag  count  \
1775             fijipro    276   
1996              camera    209   
1790  goskateboardingday    175   
1949               video    149   
1934               deals    108   
1425              liveme     96   
1435            onairnow     96   
508          3kcashprize     94   
1077               mount     86   
1706             „Ç¢„ÇØ„Ç∑„Éß„É≥„Ç´„É†     84   

                                                 corpus  
1775  Surf Restaurants with @jordysmith88‚Äã for his #...  
1996  https://t.co/2Dz0KfUMwX #GoPro Great accessory...  
1790  RT @GoPro: Video of the Day! Stoked to announc...  
1949  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
1934  #Deals #Camera Bow Archery Rifle Gun Fishing R...  
1425  #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
1435  #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
508   #Liveme #3Kcashpriz

In [41]:
df.head()

,hashtag,count,corpus,cluster,cluster_count
0,taxi,2,4 #bottle #pepsi ....\n#gari meri #taxi ....\n...,0,15
1,tachira,11,Protesta de trabajadores de la #Pepsi #Tachira...,-1,1005
2,agency,1,#Pepsi starts their #own #ContentMarketing #Ag...,-1,1005
3,summer,1,What cooler than being cool? Ice cold! \n\n#Pe...,-1,1005
4,esme,1,RT @karen73984451: #StaffySunday #Red &amp; Ma...,-1,1005


In [77]:
df[df['cluster'] == 112]

,hashtag,count,corpus,cluster,cluster_count


In [79]:
dist.shape

(2510, 2510)

In [87]:
from sklearn.metrics.pairwise import cosine_similarity
%time dist = 1 - cosine_similarity(doc_topic_matrix)
from sklearn.cluster import DBSCAN
dbs = DBSCAN(metric='precomputed')
%time dbs.fit(dist)
clusters = dbs.labels_.tolist()
df['cluster'] = clusters
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['cluster']]
group = df.groupby('cluster')['count'].aggregate(sum).sort_values(ascending=False)
top_clusters = group[:20]
for i in top_clusters.index:
    if i == -1:
        pass
    else:
        print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_clusters.ix[i])
        print df[['hashtag','count','corpus']][df.cluster == i].sort_values('count',ascending=False)[:10]
        print

CPU times: user 170 ms, sys: 48.1 ms, total: 218 ms
Wall time: 131 ms
CPU times: user 138 ms, sys: 14.3 ms, total: 152 ms
Wall time: 77.6 ms
Cluster #: 0 Total Tweeters: 11785
                 hashtag  count  \
2297               gopro   2762   
1775             fijipro    276   
1996              camera    209   
1790  goskateboardingday    175   
1949               video    149   
1934               deals    108   
1435            onairnow     96   
1425              liveme     96   
508          3kcashprize     94   
1077               mount     86   

                                                 corpus  
2297  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
1775  Surf Restaurants with @jordysmith88‚Äã for his #...  
1996  https://t.co/2Dz0KfUMwX #GoPro Great accessory...  
1790  RT @GoPro: Video of the Day! Stoked to announc...  
1949  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
1934  #Deals #Camera Bow Archery Rifle Gun Fishing R...  
1435  #Liveme #3Kcas

And once more with MeanShift

In [85]:
from sklearn.cluster import MeanShift
ms = MeanShift()
%time ms.fit(doc_topic_matrix)
clusters = ms.labels_.tolist()
df['cluster'] = clusters
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['cluster']]
group = df.groupby('cluster')['count'].aggregate(sum).sort_values(ascending=False)
top_clusters = group[:20]
for i in top_clusters.index:
    if i == -1:
        pass
    else:
        print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_clusters.ix[i])
        print df[['hashtag','count','corpus']][df.cluster == i].sort_values('count',ascending=False)[:10]
        print

CPU times: user 18.3 s, sys: 956 ms, total: 19.2 s
Wall time: 19.6 s
Cluster #: 0 Total Tweeters: 6326
                 hashtag  count  \
1775             fijipro    276   
1790  goskateboardingday    175   
1949               video    149   
1425              liveme     96   
1435            onairnow     96   
508          3kcashprize     94   
1706             „Ç¢„ÇØ„Ç∑„Éß„É≥„Ç´„É†     84   
933                scuba     76   
2062             virales     72   
684               hawaii     72   

                                                 corpus  
1775  Surf Restaurants with @jordysmith88‚Äã for his #...  
1790  RT @GoPro: Video of the Day! Stoked to announc...  
1949  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
1425  #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
1435  #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
508   #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
1706  POV!! Rhodes2016 by Rob Mcquaide https://t.co/...  
933   Would you call this 

In [84]:
from sklearn.metrics.pairwise import cosine_similarity
%time dist = 1 - cosine_similarity(doc_topic_matrix)
ms = MeanShift()
%time ms.fit(doc_topic_matrix)
clusters = ms.labels_.tolist()
df['cluster'] = clusters
df['cluster_count'] = [df['count'][df['cluster'] == row].sum() for row in df['cluster']]
group = df.groupby('cluster')['count'].aggregate(sum).sort_values(ascending=False)
top_clusters = group[:20]
for i in top_clusters.index:
    if i == -1:
        pass
    else:
        print 'Cluster #: ' + str(i) + ' Total Tweeters: ' + str(top_clusters.ix[i])
        print df[['hashtag','count','corpus']][df.cluster == i].sort_values('count',ascending=False)[:10]
        print

CPU times: user 171 ms, sys: 46.7 ms, total: 218 ms
Wall time: 129 ms
CPU times: user 18.2 s, sys: 967 ms, total: 19.2 s
Wall time: 19.2 s
Cluster #: 0 Total Tweeters: 6326
                 hashtag  count  \
1775             fijipro    276   
1790  goskateboardingday    175   
1949               video    149   
1425              liveme     96   
1435            onairnow     96   
508          3kcashprize     94   
1706             „Ç¢„ÇØ„Ç∑„Éß„É≥„Ç´„É†     84   
933                scuba     76   
2062             virales     72   
684               hawaii     72   

                                                 corpus  
1775  Surf Restaurants with @jordysmith88‚Äã for his #...  
1790  RT @GoPro: Video of the Day! Stoked to announc...  
1949  M≈Øj neigbourhood üòéüöÅ\n#aerial #video #quadroc...  
1425  #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
1435  #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
508   #Liveme #3Kcashprize #gopro giveaway Talin Sil...  
1706  POV!! Rh

Notes:
- 6/15 Used textacy and setup LSA, but it doesn't seem to return anything actionable, going to try clustering by cosine similarity

Connect tweets, tweeter counts, work on summarizer 

add user ids for direct marketing